In [ ]:
#libraries 
import importlib
from youtube_api.ytchannel import Ytchannel
from youtube_api.ytvideo import Ytvideo
from nlp.nlp_ner import NER
from etl.etlner import EtlNER
from youtube_api.transcribe import Transcribe
from nlp.nlp_sa import SA
from etl.etlsa import EtlSA
from etl.etldf import EtlDF
import pandas as pd
from etl.dbmanager import DBmanager
from sqlalchemy import create_engine
from nlp.nlp_summarisation import Summariser



In [ ]:

# initialisation

#initialising
channel_manage = Ytchannel()
video_manage = Ytvideo()
nlp_ner = NER()
nlp_sa = SA()
etlner_manage = EtlNER()
scribe_manage = Transcribe()
etlsa_manage = EtlSA()
etldf = EtlDF()
dbm = DBmanager()
sums = Summariser()

id = 'UCVjlpEjEY9GpksqbEesJnNA'

In [ ]:
""" testing 1 video """
# meta
location = channel_manage.get_videos(id)
meta = video_manage.export_info(location)


#transcribe
captions = scribe_manage.get_captions(meta,1)

# summarisation
sum_data = sums.summarise(captions,1)

#sa
sa_raw = nlp_sa.sa(captions,1)
sa_data = etlsa_manage.direct_etl(sa_raw)

#ner
ner_raw = nlp_ner.ner(captions,1)
ner_data = etlner_manage.direct_etl(ner_raw)


In [ ]:
# convert to dataframe 
mdf = etldf.meta_dataframe(meta)
sadf = etldf.sa_dataframe(sa_data)
nerdf = etldf.ner_dataframe(ner_data)
wdf = etldf.words_dataframe(captions,sum_data)

In [ ]:
# upload
dbm.upload_df(mdf,'channel_data')
dbm.upload_df(sadf,'sa_data')
dbm.upload_df(wdf,'words_data')
dbm.upload_nerdf(nerdf)